# Analyse de données et modèle de prédiction

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

### Initialize Spark session

In [2]:
spark = SparkSession.builder \
    .appName("Vehicle Category Prediction") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
def print_df(df):
    print("Size", df.count())
    df.printSchema()
    df.show(5)

def load_data_in_csv_file(file_path, delimit = ",", head=True):
    df = spark.read.options(delimiter=",", header=True, inferSchema=True).csv(file_path)

    # Display schema and first five rows of the DataFrame
    print("Schema and first rows in", file_path)
    print_df(df)
    
    return df

### Load the dataset

In [4]:
# File path in Hadoop FS
hdfs_path = "hdfs:///tpa_groupe_14/data/fusion/fusion.csv"

# Load CSV data into a DataFrame
vehicles_df = load_data_in_csv_file(hdfs_path)

Schema and first rows in hdfs:///tpa_groupe_14/data/fusion/fusion.csv


Size 100306
root
 |-- client_view.id: integer (nullable = true)
 |-- client_view.age: integer (nullable = true)
 |-- client_view.sexe: string (nullable = true)
 |-- client_view.taux: integer (nullable = true)
 |-- client_view.situationfamiliale: string (nullable = true)
 |-- client_view.nbenfantsacharge: integer (nullable = true)
 |-- client_view.deuxiemevoiture: integer (nullable = true)
 |-- immatriculation_co2_view.categorie: string (nullable = true)

+--------------+---------------+----------------+----------------+------------------------------+----------------------------+---------------------------+----------------------------------+
|client_view.id|client_view.age|client_view.sexe|client_view.taux|client_view.situationfamiliale|client_view.nbenfantsacharge|client_view.deuxiemevoiture|immatriculation_co2_view.categorie|
+--------------+---------------+----------------+----------------+------------------------------+----------------------------+---------------------------+-------

### Data formatting

In [5]:
column_names = vehicles_df.schema.names

for column_name in column_names:
    vehicles_df = vehicles_df.withColumnRenamed(column_name, column_name.split(".")[-1])

print_df(vehicles_df)

Size 100306
root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- sexe: string (nullable = true)
 |-- taux: integer (nullable = true)
 |-- situationfamiliale: string (nullable = true)
 |-- nbenfantsacharge: integer (nullable = true)
 |-- deuxiemevoiture: integer (nullable = true)
 |-- categorie: string (nullable = true)

+---+---+----+----+------------------+----------------+---------------+---------+
| id|age|sexe|taux|situationfamiliale|nbenfantsacharge|deuxiemevoiture|categorie|
+---+---+----+----+------------------+----------------+---------------+---------+
|  2| 77|   M| 971|         EN_COUPLE|               2|              0|Familiale|
| 13| 53|   M| 237|         EN_COUPLE|               1|              0| Citadine|
| 13| 53|   M| 237|         EN_COUPLE|               1|              0|     Luxe|
| 18| 84|   M| 402|       CELIBATAIRE|               0|              0|     Luxe|
| 26| 47|   M| 213|         EN_COUPLE|               0|              0|Famil

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer

### Optional

In [7]:
# Optional: Sub-sample the DataFrame
vehicles_df = vehicles_df.sample(withReplacement=False, fraction=0.3, seed=42)

In [9]:
# Prepare features
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_index").fit(vehicles_df)
    for column in ["sexe", "situationfamiliale", "categorie"]
]

assembler = VectorAssembler(
    inputCols=[
        "age", "taux", "nbenfantsacharge", "deuxiemevoiture",
        "sexe_index", "situationfamiliale_index"
    ],
    outputCol="features"
)

# Split the DataFrame into training and test sets (80% training, 20% test)
train_df, test_df = vehicles_df.randomSplit([0.8, 0.2], seed=42)

pipeline = Pipeline(stages=indexers + [assembler])
pipeline_model = pipeline.fit(train_df)

train_df = pipeline_model.transform(train_df)
test_df = pipeline_model.transform(test_df)

In [10]:
label_predictions = test_df.select("categorie", "categorie_index").distinct().rdd.collect()
categorie_dict = {row['categorie_index']: row['categorie'] for row in label_predictions}

print(categorie_dict)

{0.0: 'Luxe', 1.0: 'Familiale', 3.0: 'Citadine', 2.0: 'Compacte'}


In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import MulticlassMetrics

# La précision & Matrice de confusion

In [12]:
# Evaluation function
def evaluate_classifier(classifier, param_grid, train_df, test_df, title):
    evaluator = MulticlassClassificationEvaluator(labelCol="categorie_index", predictionCol="prediction", metricName="accuracy")
    
    # Create the cross-validator
    cross_validator = CrossValidator(
        estimator=classifier,
        estimatorParamMaps=param_grid,
        evaluator=evaluator,
        numFolds=3, 
        seed=42
    )
    
    # Train the model with the best hyperparameters
    cv_model = cross_validator.fit(train_df)
    
    # Make predictions on the test data
    predictions = cv_model.transform(test_df)
    
    # Evaluate the model
    accuracy = evaluator.evaluate(predictions)
    
    # Calcul de la matrice de confusion
    predictions_and_labels = predictions.select("prediction", "categorie_index")
    prediction_rdd = predictions_and_labels.rdd.map(tuple)
    
    metrics = MulticlassMetrics(prediction_rdd)
    confusion_matrix = metrics.confusionMatrix().toArray()

    # Calcul des taux de succès par classe
    labels = predictions.select("categorie_index").distinct().rdd.flatMap(lambda x: x).collect()
    
    per_class_metrics = {}
    for label in labels:
        per_class_metrics[label] = {
            "precision": metrics.precision(label),
            "recall": metrics.recall(label),
            "f1_score": metrics.fMeasure(label)
        }
    
    # Calcul des métriques globales pondérées
    weighted_metrics = {
        "precision": metrics.weightedPrecision,
        "recall": metrics.weightedRecall,
        "f1_score": metrics.weightedFMeasure(),
        "accuracy": accuracy
    }

    print(title + " Accuracy = {:.2f}".format(accuracy))
    print("\nWeighted Metrics:")
    print(f"  Weighted Precision: {weighted_metrics['precision']:.2f}")
    print(f"  Weighted Recall: {weighted_metrics['recall']:.2f}")
    print(f"  Weighted F1 Score: {weighted_metrics['f1_score']:.2f}")
    print(f"  Accuracy: {weighted_metrics['accuracy']:.2f}")
    
    return {
        "classifier": title,
        "cv_model": cv_model,
        "accuracy": accuracy,
        "confusion_matrix": confusion_matrix,
        "per_class_metrics": per_class_metrics,
        "weighted_metrics": weighted_metrics
    }

In [13]:
from pyspark.ml.classification import (
    DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, 
    LinearSVC, LogisticRegression, MultilayerPerceptronClassifier, NaiveBayes, OneVsRest
)

In [14]:
def calculate_composite_score(metrics, weights):
    score = 0
    
    for metric, value in metrics.items():
        score += weights.get(metric, 0) * value
        
    return score

In [15]:
def compare_classifiers(classifiers, param_grids, train_df, test_df, titles):
    results = []
    
    for classifier, param_grid, title in zip(classifiers, param_grids, titles):
        print(f"\nEvaluating {title}...\n")
        result = evaluate_classifier(classifier, param_grid, train_df, test_df, title)
        weighted_metrics = result['weighted_metrics']
        composite_score = calculate_composite_score(weighted_metrics, weighted_metrics)
        results.append({ "composite_score": composite_score, "cv_model": result["cv_model"], "classifier": result["classifier"] })
    
    # Rank the models based on composite scores
    ranked_results = sorted(results, key=lambda item: item["composite_score"], reverse=True)

    print("Model with highest score:", ranked_results[0]["classifier"], "[", ranked_results[0]["composite_score"], "]")
    
    return ranked_results[0]["cv_model"]

## Entrainement et comparaison des modèles

In [16]:
rf = RandomForestClassifier(labelCol="categorie_index", featuresCol="features")

# Define the hyperparameter grid
rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

dt = DecisionTreeClassifier(labelCol="categorie_index", featuresCol="features")

dt_param_grid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10]) \
    .addGrid(dt.impurity, ["gini", "entropy"]) \
    .build()

gbt = GBTClassifier(labelCol="categorie_index", featuresCol="features")
gbt_ovr = OneVsRest(classifier=gbt, labelCol="categorie_index", featuresCol="features")

gbt_param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [10, 20]) \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .build()

svm = LinearSVC(labelCol="categorie_index", featuresCol="features")
svm_ovr = OneVsRest(classifier=svm, labelCol="categorie_index", featuresCol="features")

svm_param_grid = ParamGridBuilder() \
    .addGrid(svm.maxIter, [10, 20]) \
    .addGrid(svm.regParam, [0.01, 0.1]) \
    .build()

lr = LogisticRegression(labelCol="categorie_index", featuresCol="features")

lr_param_grid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [10, 20]) \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

num_classes = train_df.select("categorie_index").distinct().count()

mlp = MultilayerPerceptronClassifier(labelCol="categorie_index", featuresCol="features", layers=[6, 5, 4, num_classes])

mlp_param_grid = ParamGridBuilder() \
    .addGrid(mlp.maxIter, [50, 100]) \
    .build()

nb = NaiveBayes(labelCol="categorie_index", featuresCol="features")

nb_param_grid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.5, 1.0, 1.5]) \
    .build()

classifiers = [
    rf,
    dt, 
    gbt_ovr,
    svm_ovr,
    lr,
    mlp,
    nb
]

param_grids = [
    rf_param_grid,
    dt_param_grid, 
    gbt_param_grid,
    svm_param_grid,
    lr_param_grid,
    mlp_param_grid,
    nb_param_grid
]

titles = [
    "RandomForestClassifier",
    "DecisionTreeClassifier", 
    "GBTClassifier",
    "LinearSVC",
    "LogisticRegression",
    "MultilayerPerceptronClassifier",
    "NaiveBayes"
]

# Assuming train_df and test_df are already defined
model = compare_classifiers(classifiers, param_grids, train_df, test_df, titles)


Evaluating RandomForestClassifier...



24/06/09 20:23:12 WARN DAGScheduler: Broadcasting large task binary with size 1093.9 KiB
24/06/09 20:23:13 WARN DAGScheduler: Broadcasting large task binary with size 1577.8 KiB
24/06/09 20:23:33 WARN DAGScheduler: Broadcasting large task binary with size 1132.1 KiB
24/06/09 20:23:34 WARN DAGScheduler: Broadcasting large task binary with size 1651.5 KiB
24/06/09 20:23:35 WARN DAGScheduler: Broadcasting large task binary with size 1010.7 KiB
24/06/09 20:23:53 WARN DAGScheduler: Broadcasting large task binary with size 1078.7 KiB
24/06/09 20:23:54 WARN DAGScheduler: Broadcasting large task binary with size 1562.2 KiB
24/06/09 20:24:04 WARN DAGScheduler: Broadcasting large task binary with size 1161.8 KiB
24/06/09 20:24:05 WARN DAGScheduler: Broadcasting large task binary with size 1721.4 KiB
24/06/09 20:24:07 WARN DAGScheduler: Broadcasting large task binary with size 1013.8 KiB
/home/vagrant/.local/lib/python3.9/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.

RandomForestClassifier Accuracy = 0.72

Weighted Metrics:
  Weighted Precision: 0.74
  Weighted Recall: 0.72
  Weighted F1 Score: 0.72
  Accuracy: 0.72

Evaluating DecisionTreeClassifier...



/home/vagrant/.local/lib/python3.9/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


DecisionTreeClassifier Accuracy = 0.73

Weighted Metrics:
  Weighted Precision: 0.77
  Weighted Recall: 0.73
  Weighted F1 Score: 0.73
  Accuracy: 0.73

Evaluating GBTClassifier...



24/06/09 20:25:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/09 20:26:07 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/06/09 20:27:00 WARN DAGScheduler: Broadcasting large task binary with size 1002.7 KiB
24/06/09 20:27:00 WARN DAGScheduler: Broadcasting large task binary with size 1038.5 KiB
24/06/09 20:27:00 WARN DAGScheduler: Broadcasting large task binary with size 1033.2 KiB
24/06/09 20:27:00 WARN DAGScheduler: Broadcasting large task binary with size 1033.7 KiB
24/06/09 20:27:01 WARN DAGScheduler: Broadcasting large task binary with size 1034.2 KiB
24/06/09 20:27:01 WARN DAGScheduler: Broadcasting large task binary with size 1035.5 KiB
24/06/09 20:27:01 WARN DAGScheduler: Broadcasting large task binary with size 1037.9 KiB
24/06/09 20:27:01 WARN DAGScheduler: Broadcasting large task binary with size 1042.1 KiB
24/06/09 20:27:01 WARN DAGScheduler: Broadcasting large task binary with size 1049.3 KiB
24/06/

GBTClassifier Accuracy = 0.73

Weighted Metrics:
  Weighted Precision: 0.75
  Weighted Recall: 0.73
  Weighted F1 Score: 0.72
  Accuracy: 0.73

Evaluating LinearSVC...



LinearSVC Accuracy = 0.62

Weighted Metrics:
  Weighted Precision: 0.55
  Weighted Recall: 0.62
  Weighted F1 Score: 0.58
  Accuracy: 0.62

Evaluating LogisticRegression...

LogisticRegression Accuracy = 0.64

Weighted Metrics:
  Weighted Precision: 0.62
  Weighted Recall: 0.64
  Weighted F1 Score: 0.62
  Accuracy: 0.64

Evaluating MultilayerPerceptronClassifier...



24/06/09 20:36:00 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
24/06/09 20:36:00 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25


MultilayerPerceptronClassifier Accuracy = 0.54

Weighted Metrics:
  Weighted Precision: 0.47
  Weighted Recall: 0.54
  Weighted F1 Score: 0.48
  Accuracy: 0.54

Evaluating NaiveBayes...

NaiveBayes Accuracy = 0.47

Weighted Metrics:
  Weighted Precision: 0.55
  Weighted Recall: 0.47
  Weighted F1 Score: 0.47
  Accuracy: 0.47
Model with highest score: DecisionTreeClassifier [ 2.1887612461271955 ]


# Prediction

- Charger Marketing

In [17]:
from pyhive import hive
import pandas as pd

In [18]:
hive_host = 'localhost'
hive_port = 10000
hive_username = ' ' 
hive_password = ' '

In [19]:
#conn = None
try:
    # Établir une connexion avec authentification LDAP
    conn = hive.Connection(
        host=hive_host,
        port=hive_port,
        username=hive_username,
        password=hive_password,
        auth='LDAP'  
    )
    print("Connecté à Hive avec succès")
except Exception as e:
    print(f"Erreur lors de la connexion à Hive: {e}")

Connecté à Hive avec succès


In [20]:
# Créer un curseur
cursor=conn.cursor()

In [21]:
def findAll(table) :
    # Exécuter une requête pour récupérer les données de la table "catalogue"
    query = "SELECT * FROM " + table
    cursor.execute(query)

    # Charger les résultats dans un DataFrame Pandas
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description] 
    df = spark.createDataFrame(data, schema=columns)
    # df = spark.createDataFrame(pd_df)
    
    return df

In [22]:
marketing_df = findAll("marketing_view")

Prétraiter les Données Marketing

In [23]:
# Renommer les colonnes pour éviter les conflits
for column_name in marketing_df.columns:
    marketing_df = marketing_df.withColumnRenamed(column_name, column_name.split(".")[-1])

In [24]:
marketing_df.show(5)

+---+---+----+----+------------------+----------------+---------------+
| id|age|sexe|taux|situationfamiliale|nbenfantsacharge|deuxiemevoiture|
+---+---+----+----+------------------+----------------+---------------+
|  4| 26|   F| 420|         EN_COUPLE|               3|              1|
| 20| 59|   M| 748|         EN_COUPLE|               0|              1|
| 15| 60|   M| 524|         EN_COUPLE|               0|              1|
|  2| 35|   M| 223|       CELIBATAIRE|               0|              0|
|  8| 43|   F| 431|       CELIBATAIRE|               0|              0|
+---+---+----+----+------------------+----------------+---------------+
only showing top 5 rows



Appliquer le Modèle de Prédiction

In [25]:
marketing_df = pipeline_model.transform(marketing_df)

24/06/09 20:38:27 WARN StringIndexerModel: Input column categorie does not exist during transformation. Skip StringIndexerModel for this column.


In [26]:
marketing_df.show(5)

+---+---+----+----+------------------+----------------+---------------+----------+------------------------+--------------------+
| id|age|sexe|taux|situationfamiliale|nbenfantsacharge|deuxiemevoiture|sexe_index|situationfamiliale_index|            features|
+---+---+----+----+------------------+----------------+---------------+----------+------------------------+--------------------+
|  4| 26|   F| 420|         EN_COUPLE|               3|              1|       1.0|                     0.0|[26.0,420.0,3.0,1...|
| 20| 59|   M| 748|         EN_COUPLE|               0|              1|       0.0|                     0.0|[59.0,748.0,0.0,1...|
| 15| 60|   M| 524|         EN_COUPLE|               0|              1|       0.0|                     0.0|[60.0,524.0,0.0,1...|
|  2| 35|   M| 223|       CELIBATAIRE|               0|              0|       0.0|                     1.0|[35.0,223.0,0.0,0...|
|  8| 43|   F| 431|       CELIBATAIRE|               0|              0|       1.0|               

In [27]:
predictions = model.transform(marketing_df)

In [28]:
# Sélectionner les colonnes pertinentes pour l'affichage
for key, value in categorie_dict.items():
    predictions = predictions.withColumn("prediction", when(col('prediction') == key, value).otherwise(col('prediction')))

results = predictions.select("id", "prediction")

results.show()

+---+----------+
| id|prediction|
+---+----------+
|  4|  Citadine|
| 20|      Luxe|
| 15|      Luxe|
|  2|      Luxe|
|  8|      Luxe|
| 12|      Luxe|
| 13|      Luxe|
|  6| Familiale|
| 10| Familiale|
|  1|      Luxe|
|  7| Familiale|
| 16|  Citadine|
| 17|      Luxe|
|  3|      Luxe|
|  9| Familiale|
| 14|      Luxe|
| 18|  Citadine|
|  5| Familiale|
| 19|      Luxe|
| 11| Familiale|
+---+----------+



Exporter les Résultats

In [29]:
# Chemin de sortie pour les résultats
results_hdfs_path = "hdfs:///tpa_groupe_14/results"

# Sauvegarder les résultats dans HDFS
results.write.csv(results_hdfs_path, header=True)

In [30]:
!hadoop fs -ls /tpa_groupe_14/results

Found 3 items
-rw-r--r--   1 vagrant supergroup          0 2024-06-09 20:38 /tpa_groupe_14/results/_SUCCESS
-rw-r--r--   1 vagrant supergroup        103 2024-06-09 20:38 /tpa_groupe_14/results/part-00000-1ddbf150-93de-40fb-8769-22271e2dfa5c-c000.csv
-rw-r--r--   1 vagrant supergroup        118 2024-06-09 20:38 /tpa_groupe_14/results/part-00001-1ddbf150-93de-40fb-8769-22271e2dfa5c-c000.csv
